In [1]:
import pandas as pd
from siml_data import Featureset

In [2]:
feature_loader = Featureset()

In [3]:
df = feature_loader.get()

In [5]:
df.dtypes

GCM24Open        float64
GCM24High        float64
GCM24Low         float64
GCM24Close       float64
IXICOpen         float64
IXICHigh         float64
IXICLow          float64
IXICClose        float64
HKDOpen          float64
HKDHigh          float64
HKDLow           float64
HKDClose         float64
GXCOpen          float64
GXCHigh          float64
GXCLow           float64
GXCClose         float64
CNYOpen          float64
CNYHigh          float64
CNYLow           float64
CNYClose         float64
GSPCOpen         float64
GSPCHigh         float64
GSPCLow          float64
GSPCClose        float64
CLK24Open        float64
CLK24High        float64
CLK24Low         float64
CLK24Close       float64
MCHIOpen         float64
MCHIHigh         float64
MCHILow          float64
MCHIClose        float64
0386Open         float64
0386High         float64
0386Low          float64
0386Close        float64
BLKOpen          float64
BLKHigh          float64
BLKLow           float64
BLKClose         float64
